In [2]:
#pip install super-gradients

c:\Users\Admin\github\AI_project


In [2]:
from IPython.display import clear_output
import warnings
from super_gradients.training import models
import os
import torch
warnings.filterwarnings('ignore')
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"

In [3]:
### Loading Base Model
# options : 'yolo_nas_l' ,'yolo_nas_m','yolo_nas_s'


MODEL_ARCH = 'yolo_nas_l'
model = models.get(MODEL_ARCH, pretrained_weights="coco").to(DEVICE)





The console stream is logged into C:\Users\Admin\sg_logs\console.log


[2023-11-29 15:21:04] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-11-29 15:21:04] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-11-29 15:21:05,043] torch.distributed.elastic.multiprocessing.redirects: [WARNING] NOTE: Redirects are currently not supported in Windows or MacOs.
[2023-11-29 15:21:05] INFO - utils.py - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2023-11-29 15:21:05] INFO - utils.py - NumExpr defaulting to 8 threads.
[2023-11-29 15:21:15] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-11-29 15:21:15] WARNING - export.py - Failed to import pytorch_quantization
[2023-11-29 15:21:15] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-11-29 15:21:15] WARNING - env_sanity_check.py - Failed to verify operating system: Deci officially supports only Linux kernel

In [ ]:
#####Finetunning#######

In [5]:
from super_gradients.training import Trainer
CHECKPOINT_DIR = 'checkpoints'
trainer = Trainer(experiment_name='yolonas_AI',
                  ckpt_root_dir=CHECKPOINT_DIR)


In [6]:
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val

In [7]:
dataset_params = {
    'data_dir': 'data',
    'train_images_dir': 'train/train_img',
    'train_labels_dir': 'train/trainyolo',
    'val_images_dir': 'val/val_img',
    'val_labels_dir': 'val/valyolo',
    'test_images_dir': 'test/test_img',
    'test_labels_dir': 'test/testyolo',
    'classes': ['Pedestrian', 'Cyclist', 'Car', 'Truck', 'Tram',  'Tricycle'],
}


In [10]:


train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 7,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 7,
        'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 7,
        'num_workers': 2
    }
)

clear_output()


Indexing dataset annotations: 100%|██████████| 20/20 [00:00<00:00, 54.83it/s]


In [11]:
train_data.dataset.transforms

[DetectionMosaic('additional_samples_count': 3, 'non_empty_targets': False, 'prob': 1.0, 'input_dim': (640, 640), 'enable_mosaic': True, 'border_value': 114),
 DetectionRandomAffine('additional_samples_count': 0, 'non_empty_targets': False, 'degrees': 10.0, 'translate': 0.1, 'scale': [0.1, 2], 'shear': 2.0, 'target_size': (640, 640), 'enable': True, 'filter_box_candidates': True, 'wh_thr': 2, 'ar_thr': 20, 'area_thr': 0.1, 'border_value': 114),
 DetectionMixup('additional_samples_count': 1, 'non_empty_targets': True, 'input_dim': (640, 640), 'mixup_scale': [0.5, 1.5], 'prob': 1.0, 'enable_mixup': True, 'flip_prob': 0.5, 'border_value': 114),
 DetectionHSV('additional_samples_count': 0, 'non_empty_targets': False, 'prob': 1.0, 'hgain': 5, 'sgain': 30, 'vgain': 30, 'bgr_channels': (0, 1, 2), '_additional_channels_warned': False),
 DetectionHorizontalFlip('additional_samples_count': 0, 'non_empty_targets': False, 'prob': 0.5),
 DetectionPaddedRescale('additional_samples_count': 0, 'non_em

In [12]:
#calling the pretrained model
from super_gradients.training import models
model = models.get('yolo_nas_l',
                   num_classes= len(dataset_params['classes']),
                   pretrained_weights="coco"
                   )


[2023-11-29 15:23:20] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-11-29 15:23:21] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


In [15]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models": True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": 50,
    "mixed_precision": False,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls = len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}


In [16]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)


[2023-11-29 15:23:44] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20231129_152344_166402`
[2023-11-29 15:23:44] INFO - sg_trainer.py - Checkpoints directory: checkpoints\yolonas_AI\RUN_20231129_152344_166402
[2023-11-29 15:23:44] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}


The console stream is now moved to checkpoints\yolonas_AI\RUN_20231129_152344_166402/console_Nov29_15_23_44.txt


c:\Users\Admin\anaconda3\envs\mix_env\lib\site-packages\super_gradients\common\registry\registry.py:72: DeprecationWarning: Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.
  warnings.warn(f"Object name `{name}` is now deprecated. Please replace it with `{deprecated_names[name]}`.", DeprecationWarning)
c:\Users\Admin\anaconda3\envs\mix_env\lib\site-packages\psutil\_pswindows.py:274: DeprecationWarning: getargs: The 'u' format is deprecated. Use 'U' instead.
  total, free = cext.disk_usage(path)
[2023-11-29 15:24:00] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               0          (0 available on the machine)
    - Full dataset size:            49         (len(train_set))
    - Batch size per GPU:           7          (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             7          (num_gpus * batch_si

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 2.2949
│   ├── Ppyoloeloss/loss_iou = 0.7393
│   ├── Ppyoloeloss/loss_dfl = 0.5653
│   └── Ppyoloeloss/loss = 3.5995
└── Validation
    ├── Ppyoloeloss/loss_cls = 2.3102
    ├── Ppyoloeloss/loss_iou = 0.6518
    ├── Ppyoloeloss/loss_dfl = 0.4641
    ├── Ppyoloeloss/loss = 3.426
    ├── Precision@0.50 = 0.0
    ├── Recall@0.50 = 0.0
    ├── Map@0.50 = 0.0021
    └── F1@0.50 = 0.0



Validating epoch 1: 100%|██████████| 3/3 [01:04<00:00, 21.53s/it]
[2023-11-29 15:34:41] INFO - base_sg_logger.py - Checkpoint saved in checkpoints\yolonas_AI\RUN_20231129_152344_166402\ckpt_best.pth
[2023-11-29 15:34:41] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.09453551471233368


...

SUMMARY OF EPOCH 48
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.7204
│   │   ├── Epoch N-1      = 0.7619 (↘ -0.0415)
│   │   └── Best until now = 0.7274 (↘ -0.007)
│   ├── Ppyoloeloss/loss_iou = 0.4197
│   │   ├── Epoch N-1      = 0.4493 (↘ -0.0296)
│   │   └── Best until now = 0.43   (↘ -0.0103)
│   ├── Ppyoloeloss/loss_dfl = 0.3777
│   │   ├── Epoch N-1      = 0.3867 (↘ -0.009)
│   │   └── Best until now = 0.3687 (↗ 0.009)
│   └── Ppyoloeloss/loss = 1.5178
│       ├── Epoch N-1      = 1.5979 (↘ -0.08)
│       └── Best until now = 1.5261 (↘ -0.0083)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.8558
    │   ├── Epoch N-1      = 0.8631 (↘ -0.0074)
    │   └── Best until now = 0.8137 (↗ 0.0421)
    ├── Ppyoloeloss/loss_iou = 0.5051
    │   ├── Epoch N-1      = 0.5035 (↗ 0.0016)
    │   └── Best until now = 0.497  (↗ 0.0081)
    ├── Ppyoloeloss/loss_dfl = 0.4139
    │   ├── Epoch N-1      = 0.4132 (↗ 0.0008)
    │   └── Best until now = 0.4047 (↗ 0.0093)
    ├── Ppyoloeloss/loss = 1.774

Validating epoch 49: 100%|██████████| 3/3 [00:51<00:00, 17.13s/it]


SUMMARY OF EPOCH 49
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.7503
│   │   ├── Epoch N-1      = 0.7204 (↗ 0.0299)
│   │   └── Best until now = 0.7204 (↗ 0.0299)
│   ├── Ppyoloeloss/loss_iou = 0.4707
│   │   ├── Epoch N-1      = 0.4197 (↗ 0.051)
│   │   └── Best until now = 0.4197 (↗ 0.051)
│   ├── Ppyoloeloss/loss_dfl = 0.3887
│   │   ├── Epoch N-1      = 0.3777 (↗ 0.011)
│   │   └── Best until now = 0.3687 (↗ 0.02)
│   └── Ppyoloeloss/loss = 1.6097
│       ├── Epoch N-1      = 1.5178 (↗ 0.0918)
│       └── Best until now = 1.5178 (↗ 0.0918)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.8588
    │   ├── Epoch N-1      = 0.8558 (↗ 0.0031)
    │   └── Best until now = 0.8137 (↗ 0.0452)
    ├── Ppyoloeloss/loss_iou = 0.506
    │   ├── Epoch N-1      = 0.5051 (↗ 0.0009)
    │   └── Best until now = 0.497  (↗ 0.009)
    ├── Ppyoloeloss/loss_dfl = 0.4119
    │   ├── Epoch N-1      = 0.4139 (↘ -0.0021)
    │   └── Best until now = 0.4047 (↗ 0.0072)
    ├── Ppyoloeloss/loss = 1.7767
    │  

[2023-11-29 19:21:02] INFO - sg_trainer.py - RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...
Validating epoch 50: 100%|██████████| 3/3 [01:02<00:00, 20.85s/it]


[2023-11-29 19:22:07] INFO - base_sg_logger.py - [CLEANUP] - Successfully stopped system monitoring process
